## Validação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Aplicação de características;
- Criação do vetor de características;
- Normalização de dados;
- Seleção de características;
- Classificação dos dados.

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

#### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

#### Domínio da frequência

1. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    > $\sum_{j=1}^{M}f_j PSD_j / \sum_{j=1}^{M}PSD_j$
    
    > $ f_j = j * SampleRate / 2 * M $

3. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    > $ \sum_{j=1}^{M}f_jAj / \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Tarefa 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

#### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características `VAR` & `RMS` (domínio do tempo) e `FDM` & `MMDF` (domínio da frequência).

In [19]:
from math import prod
import numpy as np
import scipy.signal

# X dominio do tempo
# W dominio da frequencia

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def log_det(x):
    from math import e
    return e ** (np.sum(np.log10(np.abs(x)), axis=-1) / np.prod(x.shape))

# funções de extração de características

# domínio do tempo:
# ************ nao ta funcionando o que ta comentado **************
# Característica WAMP - Willison Amplitude
# def wamp(signal, threshold):
#     return np.sum(np.diff(signal > threshold))
# 
# # Característica WL - Waveform length
# def wl(signal):
#     return np.sum(np.abs(np.diff(signal)))
# 
# # Característica ZC - Zero crossing
# def zc(signal, threshold=0):
#     return ((signal[:-1] > threshold) != (signal[1:] > threshold)).sum()

# Característica VAR - Variance of EMG
def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

# Característica RMS - Root Mean Square
def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

# domínio da frequência:

# Característica FMN - Primeiro momento normalizado
# ************ nao ta funcionando o que ta comentado **************
# def fmn(signal, fs):
#     freqs, psd = scipy.signal.welch(signal, fs)
#     return np.sum(freqs*psd) / np.sum(psd)
# 
# # Característica MMNF - Primeiro momento normalizado médio
# def mmnf(signal, fs):
#     freqs, psd = scipy.signal.welch(signal, fs)
#     return np.sum(freqs*psd) / len(psd)

# Característica FMD - Median Frequency
def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

# Característica MMDF - Modified Median Frequency
def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2



**Tarefa 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.


#### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

#### Implementação do vetor



In [25]:
# É pra implementar pros dois, mas por enquanto vou ir so com um pra n ter problemas
x = np.load("datasets/participant_0_time.npy")
w = np.load("datasets/participant_0_freq.npy")

print("Shape dos vetores originais (T) (F): ", x.shape, w.shape)

# aplicando caracteristicas

# Tempo
data_var = var(x=x)
data_rms = rms(x=x)

# Frequencia
data_fmd = fmd(w=w)
data_mmdf = mmdf(w=w)

data_var.shape, data_rms.shape, data_fmd.shape, data_mmdf.shape

# Shape dos vetores originais (T) (F):  (28, 2, 33, 64) (28, 2, 33, 33)
# ((28, 2, 33), (28, 2, 33), (28, 2, 33), (28, 2, 33))

Shape dos vetores originais (T) (F):  (28, 2, 33, 64) (28, 2, 33, 33)


((28, 2, 33), (28, 2, 33), (28, 2, 33), (28, 2, 33))

In [26]:
# União do vetor de caracteristicas inicial

features = np.array([data_var, data_rms, data_fmd, data_mmdf])
# features.shape
# (4, 28, 2, 33)


# Organização das dimensoes
# Samples precisam ser a primeira linha, 56 e 36 e dps 4 e 2
# 33 é as janelas, as 56 a quantidades de classes, 4 é caracteristicas e 2 eletrodos também sao caracteristicas
# ai junta os dois
features = features.transpose(1, 3, 0, 2)
# features.shape
# (28, 33, 4, 2)

# criar o vetor de caracteristicas definitivo
features = features.reshape(features.shape[0] * features.shape[1],
                            features.shape[2] * features.shape[3])
features.shape
# (924, 8)

(924, 8)

**Tarefa 3**: Aplicar normalização dos dados utilizando ferramentas ja conhecidas

In [28]:
# aplicação de normalização
from sklearn.preprocessing import StandardScaler

# Criando o Standard
scaler = StandardScaler()

# Ajustar e transformar os dados
normalized_features = scaler.fit_transform(features)

# Para novos dados de teste chamados test_features
normalized_test_features = scaler.transform(test_features)


NameError: name 'test_features' is not defined

**Tarefa 4**: Aplicação seleção de caracteristicas utilizando ferramentas ja conhecidas

In [17]:
# aplicação da seleção de caracteristicas

# codigo

**Tarefa 5**: Aplicar classificação usando 'svm'

In [ ]:
# aplicação de classificação

# codigo

## Criação do vetor de *labels*

In [18]:
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
              'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
              'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
              'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando em numerico
lab_dict = {'dir':0, 'esq': 1, 'cima':2, 'baixo':3}
labels_num = [lab_dict[item] for item in labels_str]

y = labels_num * int(features.shape[0]/len(labels_num))
len(y)
# 924 / 28
# 924/8, len(labels_num)

924